In [ ]:
from lmptools.dump import DumpFileIterator
from lmptools.polymers import Polymer
from lmptools.atom import Atom

In [ ]:
dfile = DumpFileIterator("/storage/polymer_melts_simulations_raw_datasets/one_epsilon/phi0_r0_p1000/run1/dump.flexChains.lammpstrj", unwrap=True)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
radius_of_gyration = []
for snapshot in dfile:
    for polymer in np.split(snapshot.dataframe, 1000):
        atoms = []
        polymer.apply(lambda x: atoms.append(Atom(**x.to_dict())), axis=1)
        radius_of_gyration.append(Polymer(atoms=atoms).rg)

In [ ]:
np.mean(radius_of_gyration)